In [1]:
import sys

sys.path.append("/home/td-user/rtrec")

# Movielens 1M

In [2]:
from rtrec.experiments.datasets import load_dataset

df = load_dataset(name='movielens_1m')

Using existing ratings.dat file.


In [3]:
df[['user', 'item', 'tstamp', 'rating']]

,user,item,tstamp,rating
0,1,1193,978300760,5
1,1,661,978302109,3
2,1,914,978301968,3
3,1,3408,978300275,4
4,1,2355,978824291,5
...,...,...,...,...
1000204,6040,1091,956716541,1
1000205,6040,1094,956704887,5
1000206,6040,562,956704746,5
1000207,6040,1096,956715648,4


In [4]:
# from rtrec.experiments.split import leave_one_last_item
# train_df, test_df = leave_one_last_item(df)

from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df)

#from rtrec.experiments.split import random_split
#train_df, test_df = random_split(df)

In [5]:
train_df

,user,item,rating,tstamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977
...,...,...,...,...
797753,736,1278,4,1045711206
797754,736,3671,4,1045711217
797755,3840,1196,3,1046106127
797756,3840,1225,3,1046106162


# SLIM (Sparse Linear Methods)

In [6]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

#model = SLIM()
# nn_feature_selection is fast
model = SLIM(nn_feature_selection=50) 

recommender = Recommender(model)

In [7]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 13.48it/s]

Fit completed in 4.22 seconds
Throughput: 188836.29 samples/sec


In [8]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 16.00it/s]


{'precision': 0.1315231788079517,
 'recall': 0.06727778185897296,
 'f1': 0.07272194516681163,
 'ndcg': 0.1441684598606572,
 'hit_rate': 0.6344370860927152,
 'mrr': 0.28684267581204503,
 'map': 0.06884614753891508,
 'tp': 7944,
 'auc': 0.3381075370545577}

In [9]:
recommender.model._recommend(10, top_k=10, filter_interacted=True)

[1136, 595, 260, 1035, 2797, 1304, 2406, 1259, 34, 3421]

In [10]:
recommender.similar_items(query_items=[1,34], top_k=6, ret_scores=True)

[[(3114, 0.29662662744522095),
  (588, 0.20735998451709747),
  (2355, 0.1291930079460144),
  (1265, 0.11021316796541214),
  (1923, 0.0967392772436142),
  (34, 0.08396875113248825)],
 [(588, 0.10645274072885513),
  (3114, 0.10469543933868408),
  (150, 0.10286545008420944),
  (1, 0.08672025054693222),
  (2396, 0.06297563016414642),
  (1265, 0.0576939657330513)]]

## test partial_fit

In [11]:
from rtrec.experiments.split import temporal_split

test_df1, test_df2 = temporal_split(test_df, test_frac=0.5)

In [12]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

model = SLIM(nn_feature_selection=50)
recommender = Recommender(model)

In [13]:
recommender.bulk_fit(train_df, parallel=True)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 14.28it/s]

Fit completed in 4.06 seconds
Throughput: 196483.47 samples/sec


### Note test_df1 is not fit

In [14]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 14.55it/s]


{'precision': 0.14241342377722271,
 'recall': 0.06719233160136066,
 'f1': 0.07350512091203637,
 'ndcg': 0.15608917056263985,
 'hit_rate': 0.6494109246697608,
 'mrr': 0.30375616984863724,
 'map': 0.07689420160609235,
 'tp': 3989,
 'auc': 0.35208756226517735}

### Fit test_df1

In [15]:
recommender.fit(test_df1, update_interaction=True, parallel=True)

Fitting SLIMElastic in parallel: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:02<00:00, 14.84it/s]

Fit completed in 11.82 seconds
Throughput: 8561.62 samples/sec


In [16]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 14.59it/s]


{'precision': 0.1466262049268123,
 'recall': 0.07084048165464216,
 'f1': 0.07707137284633471,
 'ndcg': 0.16124616189782442,
 'hit_rate': 0.6626204926811853,
 'mrr': 0.3156412392399544,
 'map': 0.07940554155468894,
 'tp': 4107,
 'auc': 0.3584468557147956}

# LightFM (Factorization Machines)

In [17]:
from rtrec.models import LightFM

model = LightFM(no_components=10, loss="warp", epochs=20, item_alpha=0, user_alpha=0)
recommender = Recommender(model)

In [18]:
recommender.bulk_fit(train_df)

Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.62it/s]

Fit completed in 5.65 seconds
Throughput: 141294.48 samples/sec


In [19]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 146.68it/s]


{'precision': 0.1285596026490105,
 'recall': 0.06061389976834402,
 'f1': 0.06772276898292572,
 'ndcg': 0.14118485098894368,
 'hit_rate': 0.5731788079470199,
 'mrr': 0.2761269447072417,
 'map': 0.07123823876036796,
 'tp': 7765,
 'auc': 0.3117169202670039}

In [20]:
recommender.model.recommend(1, top_k=10, filter_interacted=True)

[2396, 2858, 34, 1, 912, 1265, 356, 318, 1196, 595]

In [21]:
recommender.model.recommend_batch([2, 1], top_k=10, filter_interacted=True)

[[608, 527, 296, 858, 1617, 1094, 1704, 1674, 377, 912],
 [2396, 2858, 34, 1, 912, 1265, 356, 318, 1196, 595]]

## test partial_fit

### Note test_df1 is not fit yet

In [22]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 139.05it/s]


{'precision': 0.14119957158157814,
 'recall': 0.06112266854659811,
 'f1': 0.06925533343820073,
 'ndcg': 0.1549176622671443,
 'hit_rate': 0.5894323455908604,
 'mrr': 0.296775528014372,
 'map': 0.0806901047604835,
 'tp': 3955,
 'auc': 0.32260533369601524}

### Additionally Fit test_df1

In [23]:
recommender.fit(test_df1, update_interaction=True)

Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.88it/s]

Fit completed in 3.09 seconds
Throughput: 32735.11 samples/sec


In [24]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 136.57it/s]


{'precision': 0.14412709746519126,
 'recall': 0.06270188634892761,
 'f1': 0.07095964371446573,
 'ndcg': 0.15924445227381784,
 'hit_rate': 0.5965726526240628,
 'mrr': 0.3072118517762941,
 'map': 0.08314182808141174,
 'tp': 4037,
 'auc': 0.33063747074457583}

In [25]:
recommender.similar_items(query_items=[1,34], top_k=6, ret_scores=True)

[[(1265, 0.9445789456367493),
  (34, 0.9333197474479675),
  (588, 0.9140991568565369),
  (551, 0.9022286534309387),
  (2321, 0.8983737230300903),
  (1197, 0.8969315886497498)],
 [(2396, 0.9361806511878967),
  (1, 0.9333196878433228),
  (1265, 0.9219970703125),
  (356, 0.9039642214775085),
  (39, 0.8852192759513855),
  (2291, 0.8825164437294006)]]